In [1]:
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

from fake_useragent import UserAgent

# 強制等待 (執行期間休息一下)
from time import sleep
from pprint import pprint

In [8]:
url = 'https://irs.thsrc.com.tw/IMINT/?utm_source=thsrc&utm_medium=btnlink&utm_term=booking'
my_service = webdriver.ChromeService(executable_path="D:/專案/grab/chromedriver.exe")
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
# my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
ua = UserAgent()
my_options.add_argument('--user-agent=%s' % ua.random)
driver = webdriver.Chrome(
    options = my_options,
    service = my_service,
)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

{'identifier': '2'}

In [9]:
driver.get(url=url)
btn = driver.find_element(By.CSS_SELECTOR , '#cookieAccpetBtn')
btn.click()

In [18]:
# 打開user_info.txt
info = []
with open('./book_user_info.txt' , 'r') as file:
    content = file.readlines()
    for text in content:
        if text[0] == '#':
            continue
        info.append(text.split("-")[1].strip())
station = { "1":"南港" , "2":"台北" , "3":"板橋" , "4":"桃園" , "5":"新竹" , "6":"苗栗" , "7":"台中"
            , "8":"彰化", "9":"雲林" , "10":"嘉義" , "11":"台南" , "12":"左營"}
for i in range(2):
    try:
        info[i] = station[info[i]]
    except:
        pass
info

['南港',
 '台北',
 '2024/02/10',
 '08:30',
 '1',
 '0',
 '0',
 '0',
 '0',
 'A123456789',
 '0912345678',
 'mm123456789@gmail.com']

In [19]:
# 起點、終點

start_station = driver.find_elements(By.CSS_SELECTOR , 'div.form-group>select.uk-select')[0]
end_station = driver.find_elements(By.CSS_SELECTOR , 'div.form-group>select.uk-select')[1]
# start_station.find_elements(By.CSS_SELECTOR , 'option')[1].text
# start = '台北'
# end = '板橋'
for station in start_station.find_elements(By.CSS_SELECTOR , 'option'):
    if station.text == info[0]:
        station.click()

for station in end_station.find_elements(By.CSS_SELECTOR , 'option'):
    if station.text == info[1]:
        station.click()


In [20]:
# 出發日期
datetime = info[2].split('/')
month_dict = {'01':'一月' , '02':'二月' , '03':'三月' , '04':'四月' ,
              '05':'五月' , '06':'六月' , '07':'七月' , '08':'八月' ,
              '09':'九月' , '10':'十月' , '11':'十一月' , '12':'十二月' }
datetime_trs = month_dict[datetime[1]] + " " + datetime[2] + ", " + datetime[0]
dateform = driver.find_elements(By.CSS_SELECTOR , 'div.form-group')[2]
dateform.click()
datecheck = driver.find_elements(By.CSS_SELECTOR , 'span.flatpickr-day[tabindex]')
# datecheck[0].get_attribute('aria-label')
try:
    for check in datecheck:
        if datetime_trs == check.get_attribute('aria-label'):
            check.click()
except:
    pass

In [21]:
import datetime

In [18]:
datetime.datetime.now()

datetime.datetime(2024, 1, 22, 22, 58, 25, 281550)

In [22]:
# 出發時間
# time_for_start = '16:00'
alltime = driver.find_elements(By.CSS_SELECTOR , 'select.uk-select.out-time>option')
for time_select in alltime:
    if info[3] == time_select.text:
        time_select.click()
        

In [23]:
# 票數選擇
# regular = '2'
regularslc = driver.find_elements(By.CSS_SELECTOR , 'select.uk-select')[7]

# child = '1'
childslc = driver.find_elements(By.CSS_SELECTOR , 'select.uk-select')[8]

# disable = '1'
disableslc = driver.find_elements(By.CSS_SELECTOR , 'select.uk-select')[9]

# senior = '1'
seniorslc = driver.find_elements(By.CSS_SELECTOR , 'select.uk-select')[10]

# student = '1'
studentslc = driver.find_elements(By.CSS_SELECTOR , 'select.uk-select')[11]

for slc in regularslc.find_elements(By.CSS_SELECTOR , 'option'):
    if info[4] == slc.text:
        slc.click()

for slc in childslc.find_elements(By.CSS_SELECTOR , 'option'):
    if info[5] == slc.text:
        slc.click()

for slc in disableslc.find_elements(By.CSS_SELECTOR , 'option'):
    if info[6] == slc.text:
        slc.click()

for slc in seniorslc.find_elements(By.CSS_SELECTOR , 'option'):
    if info[7] == slc.text:
        slc.click()

for slc in studentslc.find_elements(By.CSS_SELECTOR , 'option'):
    if info[8] == slc.text:
        slc.click()



In [69]:
# 檢測沒票時發生的錯誤
testerror = driver.find_element(By.CSS_SELECTOR,'li.feedbackPanelERROR')
testerror

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"li.feedbackPanelERROR"}
  (Session info: chrome=120.0.6099.225); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF652064D02+56194]
	(No symbol) [0x00007FF651FD04B2]
	(No symbol) [0x00007FF651E776AA]
	(No symbol) [0x00007FF651EC16D0]
	(No symbol) [0x00007FF651EC17EC]
	(No symbol) [0x00007FF651F04D77]
	(No symbol) [0x00007FF651EE5EBF]
	(No symbol) [0x00007FF651F02786]
	(No symbol) [0x00007FF651EE5C23]
	(No symbol) [0x00007FF651EB4A45]
	(No symbol) [0x00007FF651EB5AD4]
	GetHandleVerifier [0x00007FF6523DD5BB+3695675]
	GetHandleVerifier [0x00007FF652436197+4059159]
	GetHandleVerifier [0x00007FF65242DF63+4025827]
	GetHandleVerifier [0x00007FF6520FF029+687785]
	(No symbol) [0x00007FF651FDB508]
	(No symbol) [0x00007FF651FD7564]
	(No symbol) [0x0000

In [24]:
departure_time = driver.find_elements(By.CSS_SELECTOR , 'p.departure-time')
for slc in departure_time[:-1]:
    print(slc.text)

In [25]:
confirmbtn = driver.find_element(By.CSS_SELECTOR , 'div.uk-flex.uk-flex-right.form-ctrl-group')
confirmbtn.click()

In [82]:
# idcard = "A123456789"
# cellphone = '0912345678'
# email = 'mm12345678@gmail.com'

idslc = driver.find_element(By.CSS_SELECTOR , 'input#idNumber.uk-input')
idslc.send_keys(info[9])
cellphoneslc = driver.find_element(By.CSS_SELECTOR , 'input#mobilePhone.uk-input')
cellphoneslc.send_keys(info[10])
emailslc = driver.find_element(By.CSS_SELECTOR , "input#email.uk-input")
emailslc.send_keys(info[11])